In [11]:
import h5py
import src.starter.utils as starter
import src.starter.visualize as vis
import pandas as pd
import numpy as np
import tqdm

In [23]:
data = pd.read_csv("data_interpolated_stats.csv")

In [24]:
max_val = data['max_val'].quantile(0.95)
min_val = data['min_val'].quantile(0.01)

In [ ]:
stage='train'

In [34]:
data = pd.read_csv("{}_data_interpolated_stats.csv".format(stage))

In [35]:
crops = h5py.File("{}_interpolated_crops.hdf5".format(stage), "w")

In [36]:
SIZE = 64

In [37]:
def normalize(volume):
    # Scale to values between 0 and 1
    mxval = max_val
    mnval = min_val
    im_volume = (volume - mnval)/max(mxval - mnval, 1e-3)
    return im_volume

In [38]:
def positions(im_shape, crop_shape, pads):
        zi, xi, yi = im_shape
        zc, xc, yc = crop_shape
        zp, xp, yp = pads
        for z in [z for z in range(0, zi, zp) if z + zc <= zi]:
            for x in [x for x in range(0, xi, xp) if x + xc <= xi]:
                for y in [y for y in range(0, yi, yp) if y + yc <= yi]:
                    yield (z, x, y)

In [39]:
# 5 for local run. replace with len(data) for cloud
crops_data = []
for i in tqdm.tqdm_notebook(range(len(data))):
    row = data.iloc[i]
    case_id, z, x, y = row['case_id'], row['num_slices'], row['height'], row['width']
    im, mask = starter.load_case(case_id)
    im, mask = im.get_data(), mask.get_data()
    if im.shape[0] % SIZE == 0:
        z_pad_size = 0
    else:
        z_pad_size = SIZE - im.shape[0] % SIZE 
    
    if im.shape[1] % SIZE == 0:
        x_pad_size = 0
    else:
        x_pad_size = SIZE - im.shape[1] % SIZE 
        
    if im.shape[2] % SIZE == 0:
        y_pad_size = 0
    else:
        y_pad_size = SIZE - im.shape[2] % SIZE 
        
    im = normalize(im)
    im = np.pad(im, ((z_pad_size, 0), (x_pad_size, 0), (y_pad_size, 0)), 'constant')
    mask = np.pad(mask, ((z_pad_size, 0), (x_pad_size, 0), (y_pad_size, 0)), 'constant')
    for position in positions((int(z + z_pad_size) , int(x + x_pad_size), int(y + y_pad_size)), (64, 64, 64), (32, 32, 32)):
        z, x, y = position
        crop = mask[z:z+64, x:x+64,y:y+64]
        kid_size = np.sum(crop == 1)
        tumor_size = np.sum(crop == 2)
        crops_data.append([case_id, position, 64, 32, kid_size, tumor_size])
    crops.create_dataset(case_id, data=np.array([im, mask]))

In [40]:
crops_data = pd.DataFrame(data=crops_data, columns=["case_id", "position", "window", "padding", "kid_size", "tumor_size"])

In [41]:
crops.close()

In [42]:
crops_data.to_csv("val_interpolated_crops.csv")